# Support Vector Machines

In [1]:
import numpy as np                     # Llibreria matemÃƒ tica
import matplotlib.pyplot as plt        # Per mostrar plots
import sklearn
import sklearn.datasets as ds            # Per carregar mÃƒÂ©s facilment el dataset digits
import sklearn.model_selection as cv    # Pel Cross-validation
import sklearn.neighbors as nb           # Per fer servir el knn
%matplotlib inline              

In [2]:
import numpy as np    # Numeric and matrix computation
import pandas as pd   # Optional: good package for manipulating data 
import sklearn as sk  # Package with learning algorithms implemented

# Loading the dataset.
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data"
data = pd.read_csv("dataWithTextWordsPrep.csv", encoding='latin1', low_memory=False)

data.head()

,Unnamed: 0,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,...,evidence,great.,toe,favor,soccer,did.,wheel,shoulder,rank,expected
0,0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,...,0,0,0,0,0,0,0,0,0,0
1,1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,...,0,0,0,0,0,0,0,0,0,0
3,3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,...,0,0,0,0,0,0,0,0,0,0
4,4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data = data.drop(['Unnamed: 0', '_unit_id', '_golden', '_unit_state', '_trusted_judgments', '_last_judgment_at',
               'gender:confidence', 'profile_yn', 'profile_yn:confidence', 'created', 'description',
               'fav_number', 'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'profileimage',
               'sidebar_color', 'text', 'tweet_count', 'tweet_created', 'tweet_id', 'tweet_location',
               'retweet_count', 'tweet_coord', 'user_timezone'], axis=1);
data = data[data.gender.notnull()]
data.head()

def genderToNumeric(gender) :
    if gender == 'male' :
        return 0
    elif gender == 'female' :
        return 1
    else :
        return 2

labels = data.gender.apply(lambda g: genderToNumeric(g))
datadt = data.drop('gender', axis=1)

print(data.shape)
data.head()

(18836, 2001)


,gender,you,weather,get,my,from,me,with,that,update,...,evidence,great.,toe,favor,soccer,did.,wheel,shoulder,rank,expected
0,male,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,male,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,male,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,male,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,female,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Cross-validation: split data into training and test sets (test 30% of data)
(X_train, X_test,  y_train, y_test) = cv.train_test_split(datadt, labels, test_size=.3, random_state=1)

Notice the stratification according to labels *y* that we demand in the spliting of data. The ionosphere dataset is small and with strarification we ensure we obtain the same proportion of examples of each class in training and test sets.

**Remember**. Data should be numerical and normalized or standarized before using an SVM. 

Normalization is not strictily necessary in our dataset because almost all columns are in range -1..1 (except columns 0 and 1)

In [5]:
data.head()

,gender,you,weather,get,my,from,me,with,that,update,...,evidence,great.,toe,favor,soccer,did.,wheel,shoulder,rank,expected
0,male,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,male,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,male,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,male,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,female,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


However, let's see how to do that properly:

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#scaler = StandardScaler().fit(X_train)
scaler = MinMaxScaler(feature_range=(-1, 1)).fit(X_train)

# Apply the normalization trained in training data in both training and test sets
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Linear SVM

Let's try an SVM with default parameters. Linear means that we are not using any kernel to move the data to a higher dimensional space.

In [ ]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

csArray = [1, 5, 50, 100]
for cElem in csArray :
    print (f'For C = {cElem}')
    knc = SVC(kernel='linear', C=cElem)
    knc.fit(X_train, y_train)
    pred=knc.predict(X_test)
    print("Confusion matrix on test set:\n",sklearn.metrics.confusion_matrix(y_test, pred))
    print("\nAccuracy on test set: ",sklearn.metrics.accuracy_score(y_test, pred))
    print()

For C = 1
Confusion matrix on test set:
 [[ 769  700  346]
 [ 706 1062  263]
 [ 510  383  912]]

Accuracy on test set:  0.485400814015

For C = 5
Confusion matrix on test set:
 [[ 782  672  361]
 [ 721 1044  266]
 [ 498  390  917]]

Accuracy on test set:  0.485400814015

For C = 50
Confusion matrix on test set:
 [[ 805  644  366]
 [ 724 1034  273]
 [ 491  378  936]]

Accuracy on test set:  0.491063528579

For C = 100


Not bad results. However, the linear SVM has parameter C that has to be adjusted. We will use *GridSearch* method to find the optimal value of C like we did in a previous notebook with the k value of the KNN algorithm.  

In [ ]:
# List of C values to test. We usualy test diverse orders of magnitude
#Cs = np.logspace(-3, 11, num=15, base=10.0)
Cs = np.logspace(-3, 5, num=9, base=10.0)

param_grid = {'C': Cs}
#grid_search = GridSearchCV(LinearSVC(), param_grid, cv=10)
grid_search = GridSearchCV(SVC(kernel='linear'), param_grid, cv=10)
grid_search.fit(X_train,y_train)

# Let's plot the 10-fold cross.validation accuracy deppending on C
scores = grid_search.cv_results_['mean_test_score']
plt.semilogx(Cs,scores)
plt.show()

# Let's apply the best C parameter found to the test set
parval=grid_search.best_params_
#knc = LinearSVC(C=parval['C']) 
knc = SVC(C=parval['C'],kernel='linear')
knc.fit(X_train, y_train)
pred=knc.predict(X_test)
print("Confusion matrix on test set:\n",sklearn.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ",sklearn.metrics.accuracy_score(y_test, pred))
print("\nBest value of parameter C found: ",parval)
print("\nNumber of supports: ",np.sum(knc.n_support_), "(",np.sum(np.abs(knc.dual_coef_)==parval['C']) ,"of them have slacks)")
print("Prop. of supports: ",np.sum(knc.n_support_)/X_train.shape[0])


On this dataset, the best C for a linear SVM is 1 (that casually is also the default value for parameter C) so we don't obtain any improvement tuning the C parameter. However, in other datasets we could obtain a dramatic increase of accuracy. 

Let's see (just for fun) how the C parameter affects performance on training and test sets. 

In [ ]:
from sklearn.model_selection import validation_curve

def plot_validation_curve(parameter_values, train_scores, validation_scores):
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    validation_scores_mean = np.mean(validation_scores, axis=1)
    validation_scores_std = np.std(validation_scores, axis=1)

    plt.fill_between(parameter_values, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(parameter_values, validation_scores_mean - validation_scores_std,
                     validation_scores_mean + validation_scores_std, alpha=0.1, color="g")
    plt.plot(parameter_values, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(parameter_values, validation_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.ylim(validation_scores_mean.min() - .1, train_scores_mean.max() + .1)
    plt.legend(loc=4)


training_scores, test_scores = validation_curve(SVC(kernel='linear'), X_train, y_train, param_name="C", param_range=Cs,cv=10)
plot_validation_curve(range(len(Cs)), training_scores, test_scores)
plt.xticks(range(len(Cs)), Cs,rotation='vertical');
plt.ylim([0.6, 1])
plt.show()

Notice that from value of C=1, increasing C results in better accuracy on the training set but worse performance on the test set. This is because being too demanding on the separation of data in the training dataset, we are overfitting to it and we decrease performance in the test set. A nice picture of typical overfitting.

## Polynomial kernels

We have seen that performance with a linear SVM is Ok but not competitive with Metamethods. However, it could happen that using kernels we could even improve accuracy. We'll try first ploynomial kernel with degree 2 with default parameters.

In [8]:
from sklearn.svm import SVC

csArray = [1, 5, 50, 100]
for cElem in csArray :
    print (f'For C = {cElem}')
    knc = SVC(kernel='poly',degree =2, C=cElem) 
    knc.fit(X_train, y_train)
    pred=knc.predict(X_test)
    print("Confusion matrix on test set:\n",sklearn.metrics.confusion_matrix(y_test, pred))
    print("\nAccuracy on test set: ",sklearn.metrics.accuracy_score(y_test, pred))
    print()

For C = 1
Confusion matrix on test set:
 [[  13 1802    0]
 [   2 2026    3]
 [  11 1448  346]]

Accuracy on test set:  0.422049194833

For C = 5
Confusion matrix on test set:
 [[1086  636   93]
 [ 889 1059   83]
 [ 818  353  634]]

Accuracy on test set:  0.491771367899

For C = 50
Confusion matrix on test set:
 [[ 846  693  276]
 [ 700 1115  216]
 [ 485  398  922]]

Accuracy on test set:  0.510175190232

For C = 100
Confusion matrix on test set:
 [[ 787  725  303]
 [ 681 1118  232]
 [ 486  405  914]]

Accuracy on test set:  0.498849761104



Better performance than the obtained with a linear SVM... It could even be increased because we didn't tune the C parameter for the polynomial kernel. Let's do that now.

In [ ]:
Cs = np.logspace(-3, 11, num=15, base=10.0)

param_grid = {'C': Cs}
grid_search = GridSearchCV(SVC(kernel='poly',degree =2) , param_grid, cv=10)
grid_search.fit(X_train,y_train)

scores = grid_search.cv_results_['mean_test_score']

plt.semilogx(Cs,scores)
plt.show()

parval=grid_search.best_params_
knc = SVC(kernel='poly',degree =2,C=parval['C']) 
knc.fit(X_train, y_train)
pred=knc.predict(X_test)
print("Confusion matrix on test set:\n",sklearn.metrics.confusion_matrix(y_test, pred))
print("\nAccuracy on test set: ",sklearn.metrics.accuracy_score(y_test, pred))
print("\nBest combination of parameters found: ",parval)
print("\nNumber of supports: ",np.sum(knc.n_support_), "(",np.sum(np.abs(knc.dual_coef_)==parval['C']) ,"of them have slacks)")
print("Prop. of supports: ",np.sum(knc.n_support_)/X_train.shape[0])

Best result so far. Competitive with meta-methods. Now the best C value found is 100 and accuracy is a lot higher than with default parameters. It's always important when working with SVMs to find best parameters. 

For this C value we have a nice accuracy on the test set. Let's try what happens now with a polynomial kernel of degree 3.

In [9]:
csArray = [1, 5, 50, 100]
for cElem in csArray :
    print (f'For C = {cElem}')
    knc = SVC(kernel='poly',degree =3, C=cElem) 
    knc.fit(X_train, y_train)
    pred=knc.predict(X_test)
    print("Confusion matrix on test set:\n",sklearn.metrics.confusion_matrix(y_test, pred))
    print("\nAccuracy on test set: ",sklearn.metrics.accuracy_score(y_test, pred))
    print()

For C = 1
Confusion matrix on test set:
 [[ 183 1631    1]
 [  86 1941    4]
 [ 142 1297  366]]

Accuracy on test set:  0.440629976995

For C = 5
Confusion matrix on test set:
 [[ 980  662  173]
 [ 788 1100  143]
 [ 651  360  794]]

Accuracy on test set:  0.508582551761

For C = 50
Confusion matrix on test set:
 [[ 811  717  287]
 [ 683 1129  219]
 [ 490  396  919]]

Accuracy on test set:  0.505928154309

For C = 100
Confusion matrix on test set:
 [[ 777  727  311]
 [ 680 1112  239]
 [ 478  414  913]]

Accuracy on test set:  0.495841443992



Very bad results! That's because, again, we didn't use the optimal value for the C parameter but the default one. Let's find the best parameter.

In [ ]:
Cs = np.logspace(-3, 11, num=15, base=10.0)

param_grid = {'C': Cs}
grid_search = GridSearchCV(SVC(kernel='poly',degree =3) , param_grid, cv=10)
grid_search.fit(X_train,y_train)

scores = grid_search.cv_results_['mean_test_score']

plt.semilogx(Cs,scores)
plt.show()

parval=grid_search.best_params_
knc = SVC(kernel='poly',degree =3,C=parval['C']) 
knc.fit(X_train, y_train)
pred=knc.predict(X_test)
print("Confusion matrix on test set:\n",sklearn.metrics.confusion_matrix(y_test, pred))
print("\nBest combination of parameters found: ",parval)
print("\nAccuracy on test set: ",sklearn.metrics.accuracy_score(y_test, pred))
print("\nNumber of supports: ",np.sum(knc.n_support_), "(",np.sum(np.abs(knc.dual_coef_)==parval['C']) ,"of them have slacks)")
print("Prop. of supports: ",np.sum(knc.n_support_)/X_train.shape[0])

Confusion matrix shows only 1 more error than the confusion matrix obtained with the quadratic kernel. Given the low number of cases we have in the test set, we cannot conclude that polynomial kernel with degree 2 is better than with degree 3. But remember that when there are several classifiers with a similar performance, we should always choose the simpler one! So we will choose the quadratic polynomial kernel as best polynomial kernel, not because of performance only but because is the better combination of performance and simplicity.

## RBF Kernel

There's another possibility for the kernel: The RBF kernel. This is the default kernel in the implementation of SVMs in sklearn, so we don't need to explicitely say the kernel used. Let's try it with default parameters.

In [ ]:
csArray = [1, 5, 50, 100]
gammaArray = [0.000001, 0.001, 0.1, 1, 10]
for cElem in csArray :
    for gammaElement in gammaArray :
        print (f'For C = {cElem}')
        knc = SVC(C=cElem, gamma=gammaElement) 
        knc.fit(X_train, y_train)
        pred=knc.predict(X_test)
        print("Confusion matrix on test set:\n",sklearn.metrics.confusion_matrix(y_test, pred))
        print("\nAccuracy on test set: ",sklearn.metrics.accuracy_score(y_test, pred))
        print()

Promising performance for default parameters. But we have to search for the best parameters. In this case we have two parameters to adjust: the C parameter and the gamma parameter. We will find the best combination using the *GridSearch* method.

In [ ]:
# Values we will test for each parameter. When observin results, consider the limits of the 
# values tested and increase them if necessary 
gammas = [0.000001,0.00001, 0.0001,0.001,0.01,0.1,1,10]
Cs = np.logspace(-1, 6, num=8, base=10.0)

param_grid = {'C': Cs, 'gamma' : gammas}
grid_search = GridSearchCV(SVC(), param_grid, cv=10)
grid_search.fit(X_train,y_train)
parval=grid_search.best_params_

# We'll show in a grid, the accuracy for each combination of parameters tester
scores = grid_search.cv_results_['mean_test_score']
scores = np.array(scores).reshape(len(param_grid['C']), len(param_grid['gamma']))

plt.matshow(scores)
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(param_grid['gamma'])), param_grid['gamma'],rotation='vertical')
plt.yticks(np.arange(len(param_grid['C'])), param_grid['C'])
plt.show()
print("\nBest combination of parameters found: ",parval)